In [1]:
import sys, os
import time, json
import requests
import pymysql
import random
from datetime import date, datetime
from Crawler_Module import StockPrice_Crawler, change_date
from dateutil.relativedelta import relativedelta
current = date.today()
print('today is %s'%current)

today is 2019-01-18


In [2]:
def time_format(date):
    return datetime.strptime(date, "%Y-%m-%d")

In [3]:
def change_date(date):
    return date + relativedelta(months=1)

In [4]:
def StockPrice_Crawler(StockCode, date):
    url = 'http://www.tse.com.tw/exchangeReport/STOCK_DAY?response=json&date='+ str(date).replace('-','') + '&stockNo=' + StockCode
    try:
        return json.loads(requests.get(url).text)['data']
    except:
        pass

In [5]:
def Create_Table(code):        
    sql = """CREATE TABLE IF NOT EXISTS `%s` ( `id` int(11) NOT NULL auto_increment PRIMARY KEY, 
          `code` int(11) NOT NULL,       
          `date` date NOT NULL,
          `trade_volume` int(100) NULL,     
          `total_trade_price` float(15) NULL,    
          `open` float(15) NOT NULL,
          `high` float(15) NOT NULL,    
          `low` float(15) NOT NULL,
          `close` float(15) NOT NULL,
          `price_differ` float(15) NOT NULL,    
          `trade_records` int(11) NOT NULL);"""

    mycursor.execute(sql,(code,))
    mydb.commit()
    return 0

In [6]:
def importer(data):
    payload = {}
    payload["code"] = str(data[0])
    payload["date"] = str(data[1])
    payload["trade_volume"] = str(data[2]).replace(',','')
    payload["total_trade_price"] = str(data[3]).replace(',','')
    payload["open"] = str(data[4]).replace(',','')
    payload["high"] = str(data[5]).replace(',','')
    payload["low"] = str(data[6]).replace(',','')
    payload["close"] = str(data[7]).replace(',','')
    payload["price_differ"] = str(data[8]).replace(',','')
    payload["trade_records"] = str(data[9]).replace(',','')
    print(payload)
    sql = """INSERT INTO `%s` ( `code`, `date`, `trade_volume`, `total_trade_price`, `open`, `high`, `low`, `close`, `price_differ`, `trade_records`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

    mycursor.execute(sql,(payload["code"],payload["code"],payload["date"],payload["trade_volume"],payload["total_trade_price"],payload["open"],payload["high"],payload["low"],payload["close"],payload["price_differ"],payload["trade_records"]))
    mydb.commit()
    return 0
    

In [7]:
def updater(code):
    print('[Status] Working on Stock: {}'.format(code))
    print('[Now Processing...] Checking the Nearest Date of Existing Stock Price')
    Create_Table(code)
    
    mycursor.execute('SELECT max(date) FROM `\'%s\'`;'%code)
    max_date = mycursor.fetchone()[0]
    if max_date == None:
        max_date = datetime.date(datetime.strptime('20070101', "%Y%m%d"))
    print('[Status] The Newest Date of The Stock Price is {}'.format(max_date))
    in_date = max_date

    while in_date < current:
        print('[Now Processing...] Crawl the Stock Price: {} {}'.format(code, in_date))
        Price = StockPrice_Crawler(code, in_date)
        
        if Price:
            for data in Price:
                date = datetime.date(datetime.strptime(str(int(data[0].replace('/','')) + 19110000),'%Y%m%d'))

                new_data = [code] + [date] + data[1:]
                
                if date > max_date:
                    importer(new_data)

        time.sleep(random.uniform(20, 25))
        in_date = change_date(in_date)

In [8]:
def main(stock_code):
    # Execution part
    Begin = datetime.now()

    # stock_list
    f = open(stock_code,'r')
    code_list = list(set(f.read().split('\n')))
    f.close()
    print('[Now Processing...] Stock Price Updating Service')
    print('----------------------------------------------------')
    print('There are \"{0}\" unique stock code from the file \"{1}\"'.format(len(code_list), stock_code.split('\\')[-1]))
    print('The first 5 stock codes:')
    for index in range(5):
        print('{0}: {1}'.format(index, code_list[index]))
    while code_list:
            code = code_list.pop()
            if code:
                updater(code)

In [9]:
if __name__ == '__main__':
    # Connect to localhost db
    mydb = pymysql.connect(
      host="localhost",
      user="stock",
      passwd="stock",
      database="stock_price"
    )

    mycursor = mydb.cursor()
    file_name = 'Stock_List.txt'
    main(file_name)

[Now Processing...] Stock Price Updating Service
----------------------------------------------------
There are "799" unique stock code from the file "Stock_List.txt"
The first 5 stock codes:
0: 2454
1: 2936
2: 2371
3: 2429
4: 8473
[Status] Working on Stock: 6582
[Now Processing...] Checking the Nearest Date of Existing Stock Price
[Status] The Newest Date of The Stock Price is 2007-01-01
[Now Processing...] Crawl the Stock Price: 6582 2007-01-01
[Now Processing...] Crawl the Stock Price: 6582 2007-02-01
[Now Processing...] Crawl the Stock Price: 6582 2007-03-01
[Now Processing...] Crawl the Stock Price: 6582 2007-04-01
[Now Processing...] Crawl the Stock Price: 6582 2007-05-01
[Now Processing...] Crawl the Stock Price: 6582 2007-06-01
[Now Processing...] Crawl the Stock Price: 6582 2007-07-01
[Now Processing...] Crawl the Stock Price: 6582 2007-08-01
[Now Processing...] Crawl the Stock Price: 6582 2007-09-01
[Now Processing...] Crawl the Stock Price: 6582 2007-10-01
[Now Processing...

D:\Program Files\Anaconda\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect double value: 'X0.00' for column 'price_differ' at row 1")
  result = self._query(query)


[Now Processing...] Crawl the Stock Price: 6582 2017-07-01
{'code': '6582', 'date': '2017-07-03', 'trade_volume': '98000', 'total_trade_price': '6821000', 'open': '70.00', 'high': '70.00', 'low': '69.30', 'close': '69.40', 'price_differ': '-0.60', 'trade_records': '68'}
{'code': '6582', 'date': '2017-07-04', 'trade_volume': '72000', 'total_trade_price': '5017900', 'open': '70.00', 'high': '70.00', 'low': '69.50', 'close': '69.50', 'price_differ': '+0.10', 'trade_records': '50'}
{'code': '6582', 'date': '2017-07-05', 'trade_volume': '173000', 'total_trade_price': '11928000', 'open': '69.70', 'high': '69.70', 'low': '68.50', 'close': '69.00', 'price_differ': '-0.50', 'trade_records': '129'}
{'code': '6582', 'date': '2017-07-06', 'trade_volume': '49000', 'total_trade_price': '3381600', 'open': '69.60', 'high': '69.60', 'low': '68.80', 'close': '68.90', 'price_differ': '-0.10', 'trade_records': '36'}
{'code': '6582', 'date': '2017-07-07', 'trade_volume': '105350', 'total_trade_price': '728

[Now Processing...] Crawl the Stock Price: 6582 2017-09-01
{'code': '6582', 'date': '2017-09-01', 'trade_volume': '145000', 'total_trade_price': '9013800', 'open': '61.80', 'high': '62.70', 'low': '60.60', 'close': '62.50', 'price_differ': '+1.90', 'trade_records': '119'}
{'code': '6582', 'date': '2017-09-04', 'trade_volume': '41050', 'total_trade_price': '2545425', 'open': '62.50', 'high': '62.60', 'low': '61.60', 'close': '61.60', 'price_differ': '-0.90', 'trade_records': '40'}
{'code': '6582', 'date': '2017-09-05', 'trade_volume': '27000', 'total_trade_price': '1657700', 'open': '61.60', 'high': '61.90', 'low': '61.10', 'close': '61.20', 'price_differ': '-0.40', 'trade_records': '22'}
{'code': '6582', 'date': '2017-09-06', 'trade_volume': '58000', 'total_trade_price': '3526400', 'open': '61.20', 'high': '61.20', 'low': '60.60', 'close': '60.80', 'price_differ': '-0.40', 'trade_records': '45'}
{'code': '6582', 'date': '2017-09-07', 'trade_volume': '33000', 'total_trade_price': '20054

[Now Processing...] Crawl the Stock Price: 6582 2017-11-01
{'code': '6582', 'date': '2017-11-01', 'trade_volume': '65000', 'total_trade_price': '3429300', 'open': '53.50', 'high': '53.50', 'low': '52.50', 'close': '52.50', 'price_differ': '+1.30', 'trade_records': '51'}
{'code': '6582', 'date': '2017-11-02', 'trade_volume': '19000', 'total_trade_price': '998800', 'open': '52.50', 'high': '52.60', 'low': '52.50', 'close': '52.60', 'price_differ': '+0.10', 'trade_records': '16'}
{'code': '6582', 'date': '2017-11-03', 'trade_volume': '32920', 'total_trade_price': '1729445', 'open': '52.70', 'high': '52.70', 'low': '52.40', 'close': '52.40', 'price_differ': '-0.20', 'trade_records': '31'}
{'code': '6582', 'date': '2017-11-06', 'trade_volume': '64203', 'total_trade_price': '3309776', 'open': '52.40', 'high': '52.40', 'low': '51.00', 'close': '51.60', 'price_differ': '-0.80', 'trade_records': '50'}
{'code': '6582', 'date': '2017-11-07', 'trade_volume': '25020', 'total_trade_price': '1284342'

...

[Now Processing...] Crawl the Stock Price: 2527 2018-07-01
{'code': '2527', 'date': '2018-07-02', 'trade_volume': '292032', 'total_trade_price': '9590648', 'open': '32.90', 'high': '33.05', 'low': '32.55', 'close': '32.65', 'price_differ': '-0.25', 'trade_records': '132'}
{'code': '2527', 'date': '2018-07-03', 'trade_volume': '657500', 'total_trade_price': '20803099', 'open': '32.50', 'high': '32.55', 'low': '31.05', 'close': '31.50', 'price_differ': '-1.15', 'trade_records': '314'}
{'code': '2527', 'date': '2018-07-04', 'trade_volume': '193000', 'total_trade_price': '6110300', 'open': '31.65', 'high': '31.95', 'low': '31.30', 'close': '31.90', 'price_differ': '+0.40', 'trade_records': '104'}
{'code': '2527', 'date': '2018-07-05', 'trade_volume': '276390', 'total_trade_price': '8637573', 'open': '31.60', 'high': '31.70', 'low': '30.80', 'close': '30.90', 'price_differ': '-1.00', 'trade_records': '167'}
{'code': '2527', 'date': '2018-07-06', 'trade_volume': '609500', 'total_trade_price'

[Now Processing...] Crawl the Stock Price: 2527 2018-09-01
{'code': '2527', 'date': '2018-09-03', 'trade_volume': '335000', 'total_trade_price': '10158400', 'open': '30.65', 'high': '30.65', 'low': '30.15', 'close': '30.30', 'price_differ': ' 0.00', 'trade_records': '143'}
{'code': '2527', 'date': '2018-09-04', 'trade_volume': '182200', 'total_trade_price': '5522180', 'open': '30.30', 'high': '30.50', 'low': '30.20', 'close': '30.40', 'price_differ': '+0.10', 'trade_records': '79'}
{'code': '2527', 'date': '2018-09-05', 'trade_volume': '202574', 'total_trade_price': '6163156', 'open': '30.25', 'high': '30.60', 'low': '30.25', 'close': '30.35', 'price_differ': '-0.05', 'trade_records': '93'}
{'code': '2527', 'date': '2018-09-06', 'trade_volume': '325000', 'total_trade_price': '9956300', 'open': '30.20', 'high': '31.35', 'low': '30.20', 'close': '31.00', 'price_differ': '+0.65', 'trade_records': '136'}
{'code': '2527', 'date': '2018-09-07', 'trade_volume': '355000', 'total_trade_price': 

[Now Processing...] Crawl the Stock Price: 2527 2018-11-01
{'code': '2527', 'date': '2018-11-01', 'trade_volume': '427000', 'total_trade_price': '10338650', 'open': '23.50', 'high': '24.80', 'low': '23.45', 'close': '24.00', 'price_differ': '+0.70', 'trade_records': '227'}
{'code': '2527', 'date': '2018-11-02', 'trade_volume': '207000', 'total_trade_price': '5025600', 'open': '24.30', 'high': '24.50', 'low': '24.05', 'close': '24.45', 'price_differ': '+0.45', 'trade_records': '110'}
{'code': '2527', 'date': '2018-11-05', 'trade_volume': '154000', 'total_trade_price': '3693000', 'open': '24.15', 'high': '24.15', 'low': '23.85', 'close': '23.95', 'price_differ': '-0.50', 'trade_records': '76'}
{'code': '2527', 'date': '2018-11-06', 'trade_volume': '145000', 'total_trade_price': '3552450', 'open': '24.20', 'high': '24.80', 'low': '24.20', 'close': '24.70', 'price_differ': '+0.75', 'trade_records': '90'}
{'code': '2527', 'date': '2018-11-07', 'trade_volume': '268000', 'total_trade_price': 

[Now Processing...] Crawl the Stock Price: 2527 2019-01-01
{'code': '2527', 'date': '2019-01-02', 'trade_volume': '126002', 'total_trade_price': '2806294', 'open': '22.40', 'high': '22.45', 'low': '22.15', 'close': '22.20', 'price_differ': '-0.20', 'trade_records': '59'}
{'code': '2527', 'date': '2019-01-03', 'trade_volume': '80000', 'total_trade_price': '1800100', 'open': '22.70', 'high': '22.75', 'low': '22.20', 'close': '22.60', 'price_differ': '+0.40', 'trade_records': '49'}
{'code': '2527', 'date': '2019-01-04', 'trade_volume': '129000', 'total_trade_price': '2917950', 'open': '22.75', 'high': '22.85', 'low': '22.40', 'close': '22.80', 'price_differ': '+0.20', 'trade_records': '60'}
{'code': '2527', 'date': '2019-01-07', 'trade_volume': '132000', 'total_trade_price': '3042550', 'open': '23.20', 'high': '23.20', 'low': '22.95', 'close': '23.15', 'price_differ': '+0.35', 'trade_records': '102'}
{'code': '2527', 'date': '2019-01-08', 'trade_volume': '57000', 'total_trade_price': '130

OperationalError: (2013, 'Lost connection to MySQL server during query ([WinError 10053] 連線已被您主機上的軟體中止。)')